IAMSI -- 2020-2021
--------
*&copy; Equipe pédagogique : Gauvain Bourgne, Jean-Gabriel Ganascia, Marie-Jeanne Lesot, Christophe Marsala.*


# TME 09 : Règles d'association

<font color="RED" size="+1">**[Q]**</font> **Indiquer dans la boîte ci-dessous vos noms et prénoms**

Lenoir Romain 3670199, Cécile Giang 3530406

<font color="RED" size="+1">**[Q]**</font> **Renommer ce notebook**

Tout en haut de cette page, cliquer sur <tt>TME-09</tt> et rajouter à la suite de <tt>TME-09</tt> les noms des membres du binômes séparés par un tiret.

Par exemple, pour le binôme Luke Skywalker et Han Solo, le nom de fichier devient : <pre>TME-09-Skywalker-Solo</pre>

Penser à sauvegarder fréquemment le fichier en cours de travail :
- soit en cliquant sur l'icône "disquette"
- soit par la combinaison de touches [Ctrl]-S

## Présentation

### But de ce TME

Ce TME a pour but de réaliser une implémentation *intuitive* de l'algorithme **apriori** afin de la comparer à une version efficace de cet algorithme, puis à une implémentation de l'algorithme **fp-growth**.


#### Contenu de l'archive téléchargée

Une fois détarée, l'archive **tme-09.tgz** crée le répertoire <code>tme09</code> contenant :
- ce notebook que vous lisez en ce moment et que vous allez compléter.
- un répertoire <code>dataset</code> contenant des bases d'exemples pour tester les algorithmes
- deux fichiers exécutables (Linux): <code>apriori32</code> et <code>fpgrowth32</code> (ces 2 programmes ont été développés par Christian Borgelt, plus d'infos sur sa page web  http://www.borgelt.net/software.html). Ces programmes seront utilisés en fin de séance. Si vous n'êtes pas sous Linux, vous pouvez trouver une version pour votre OS ici:
    - <code>apriori</code> : https://borgelt.net/apriori.html
    - <code>fpgrowth</code> : https://borgelt.net/fpgrowth.html



#### Compte-rendu de la séance

Le compte-rendu de ce TME se compose de ce notebook complété par les réponses aux questions posées.

Ce compte-rendu est à poster : 
- à l'issue de la séance, un premier envoi doit être **obligatoirement** fait avec ce que vous avez réalisé
- si nécessaire, un complément peut être envoyé **au plus tard avant le début de la prochaine séance**.


<font color="RED">IMPORTANT: soumission de votre fichier final</font>

**Nom à donner au fichier à poster** : *Nom1_Nom2.ipynb* 
- *Nom1* et *Nom2* : noms des membres du binôme
- ne pas compresser ou faire une archive: envoyer le notebook tel quel, éventuellement, si vous avez d'autres fichiers à envoyer, les  joindre au message.

In [13]:
# Vérification de la version de Python utilisée:
import sys
sys.path # le path doit contenir python3.5 ou une version supérieure

['D:\\all\\travail\\m1_info\\IAMSI\\tme-09',
 'D:\\Program Files (x86)\\Anaconda\\python38.zip',
 'D:\\Program Files (x86)\\Anaconda\\DLLs',
 'D:\\Program Files (x86)\\Anaconda\\lib',
 'D:\\Program Files (x86)\\Anaconda',
 '',
 'C:\\Users\\rl24\\AppData\\Roaming\\Python\\Python38\\site-packages',
 'D:\\Program Files (x86)\\Anaconda\\lib\\site-packages',
 'D:\\Program Files (x86)\\Anaconda\\lib\\site-packages\\win32',
 'D:\\Program Files (x86)\\Anaconda\\lib\\site-packages\\win32\\lib',
 'D:\\Program Files (x86)\\Anaconda\\lib\\site-packages\\Pythonwin',
 'D:\\Program Files (x86)\\Anaconda\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\rl24\\.ipython']

## Traitement d'une base d'apprentissage


### Chargement de la base

On commence par travailler sur la base exemple du fichier <code>exemple-1.txt</code> (fourni dans le répertoire datasets). 

Ce fichier contient une transaction par ligne. Chaque transaction est composée d'un groupe d'items séparés par un espace.

On peut charger en Python ce fichier par la commande suivante (le répertoire datasets doit se trouver dans le répertoire courant) :

In [14]:
import csv
with open('datasets/exemple-1.txt', 'r') as fichier:
    lecteur = csv.reader(fichier, delimiter=' ')
    i = 0
    for ligne in lecteur:
        i += 1
        print('ligne',i,':',ligne)

ligne 1 : ['a', 'b', 'c']
ligne 2 : ['a', 'd', 'e']
ligne 3 : ['b', 'c', 'd']
ligne 4 : ['a', 'b', 'c', 'd']
ligne 5 : ['b', 'c']
ligne 6 : ['a', 'b', 'd']
ligne 7 : ['d', 'e']
ligne 8 : ['a', 'b', 'c', 'd']
ligne 9 : ['c', 'd', 'e']
ligne 10 : ['a', 'b', 'c']


<font color="RED" size="+1">**[Q]**</font>  Ecrire la fonction <code>chargeBase</code> qui prend en argument un nom de fichier, respectant le format énoncé plus haut, le lit et rend un dictionnaire dont les clés sont les numéros de ligne (une transaction) et les valeurs associées les itemsets correspondants représentés sous forme d'ensembles Python (des <code>set()</code> donc).

Ici, il est plus intéressant de représenter un itemset comme un ensemble Python plutôt que comme une liste. Il est ainsi plus facile de réaliser des comparaisons d'ensembles ou des ajouts d'éléments.

Dans le reste de ce document, on appelle **BASE** un dictionnaire de ce type.

In [66]:
def chargeBase(nomFichier, delimiteur=' '):
    with open(nomFichier, 'r') as fichier:
        lecteur = csv.reader(fichier, delimiter=delimiteur)
        base = dict()
        i = 0
        for ligne in lecteur:
            i += 1
            s= set()
            for j in ligne:
                s.add(j)
            base[i]=s
    return base

In [16]:
print("Résultat du chargement de 'exemple-1.txt', on obtient : ")
Base1 = chargeBase('datasets/exemple-1.txt')
Base1

Résultat du chargement de 'exemple-1.txt', on obtient : 


{1: {'a', 'b', 'c'},
 2: {'a', 'd', 'e'},
 3: {'b', 'c', 'd'},
 4: {'a', 'b', 'c', 'd'},
 5: {'b', 'c'},
 6: {'a', 'b', 'd'},
 7: {'d', 'e'},
 8: {'a', 'b', 'c', 'd'},
 9: {'c', 'd', 'e'},
 10: {'a', 'b', 'c'}}

On utilise la variable **Base1** dans la suite pour faire référence à cette base.

## Itemsets et support

<font color="RED" size="+1">**[Q]**</font>  Ecrire la fonction <code>noms_items</code> qui prend en argument une BASE et rend l'ensemble des items qui composent cette base.

In [17]:
def noms_items(base):
    s= set()
    for i in base.values():
        s =s.union(i)
    return s

In [18]:
print("Pour la BASE précédente :")
noms_items(Base1)

Pour la BASE précédente :


{'a', 'b', 'c', 'd', 'e'}

<font color="RED" size="+1">**[Q]**</font>  Ecrire la fonction <code>singletons</code> qui prend en argument une BASE et rend la liste des itemsets de taille 1 obtenus à partir de cette base.

*Remarque*: attention, ici on utilise une **liste** pour stocker les itemsets (un itemset est un ensemble Python) car il n'est pas possible en Python de créer des ensembles d'ensembles.


In [19]:
def singletons(base):
    s = noms_items(base)
    res = []
    for i in s:
        j = set([i])
        res.append(j)
    return res

In [20]:
print("Exemple: singletons(Base1) rend :")
singletons(Base1)

Exemple: singletons(Base1) rend :


[{'b'}, {'c'}, {'a'}, {'e'}, {'d'}]

<font color="RED" size="+1">**[Q]**</font>  Ecrire la fonction <code>comptage</code> qui, pour une BASE et un itemset donnés, rend le nombre de transactions de BASE qui contiennent cet itemset.

In [21]:
def comptage(base, item):
    res = len(base)
    for i in base.values():
        b = 0
        for j in item:
            if j not in i:
                res-=1
                b = 0
                break
        if b==1:
            continue
                
            
    return res

In [22]:
print("Comptage de l'itemset {'a','b','c'} dans la base précédente : ")
print("comptage(Base1,{'a','b','c'}) rend la valeur "+str(comptage(Base1,{'a','b','c'})))

Comptage de l'itemset {'a','b','c'} dans la base précédente : 
comptage(Base1,{'a','b','c'}) rend la valeur 4


<font color="RED" size="+1">**[Q]**</font>  Ecrire la fonction <code>support</code> qui, pour une BASE et un itemset donnés, rend le support de cet itemset dans la BASE.

In [23]:
def support(base, item):
    return comptage(base, item)/len(base)

In [24]:
print("Support de l'itemset {'a','b','c'} dans la base précédente : " \
      +str(support(Base1,{'a','b','c'})))

Support de l'itemset {'a','b','c'} dans la base précédente : 0.4


## Implémentation de l'algorithme a-priori

Dans cette partie, une implémentation de la partie de construction des itemsets fréquents de l'algorithme a-priori est réalisée. On ne s'intéresse pas dans cette question à la génération des règles d'association (mais cela peut être fait en complément).

Votre programme doit pouvoir s'appliquer aux bases fournies dans le répertoire *datasets* (éventuellement, sur au moins les 10 premiers exemples de la base mushrooms).

<font color="RED" size="+1">**[Q]**</font>  Ecrire la fonction <code>apriori_gen</code> qui prend en argument une liste d'itemsets de même longueur $k$, applique l'algorithme apriori-gen pour rendre la liste des itemsets candidats de longueurs $k+1$.


In [50]:
def apriori_gen(liste):
    res=[]
    for i in range(len(liste)):
        for j in liste[i:]:
            if liste[i]!=j:
                u = liste[i].union(j)
                if len(u)==len(j)+1 and u not in res:
                    res.append(u)
    res2=res.copy()
    for c in res:
        for i in c:
            tmp=c.copy()
            tmp.remove(i)
            if tmp not in liste and c in res2:
                res2.pop(res2.index(c))
                
                
    return res2

In [52]:
print("Exemple: apriori_gen([{'a'}, {'b'}, {'c'}, {'d'}]) rend ")
print(apriori_gen([{'a'}, {'b'}, {'c'}, {'d'}]))

print("\nExemple: apriori_gen([{'a','b'},{'a','d'},{'b','d'},{'b','c'},{'c','d'}]) rend ")
print(apriori_gen([{'a', 'b'}, {'a', 'd'}, {'b', 'd'}, {'b', 'c'}, {'c', 'd'}]))

print("\nExemple: apriori_gen([{'b', 'a', 'd'}, {'b', 'c', 'd'}]) rend ")
print(apriori_gen([{'b', 'a', 'd'}, {'b', 'c', 'd'}]))

Exemple: apriori_gen([{'a'}, {'b'}, {'c'}, {'d'}]) rend 
[{'b', 'a'}, {'a', 'c'}, {'a', 'd'}, {'b', 'c'}, {'b', 'd'}, {'c', 'd'}]

Exemple: apriori_gen([{'a','b'},{'a','d'},{'b','d'},{'b','c'},{'c','d'}]) rend 
[{'b', 'a', 'd'}, {'b', 'c', 'd'}]

Exemple: apriori_gen([{'b', 'a', 'd'}, {'b', 'c', 'd'}]) rend 
[]


<font color="RED" size="+1">**[Q]**</font>  Ecrire la fonction <code>apriori</code> qui prend en argument une BASE et une valeur réelle comprise entre 0 et 1, et qui rend une liste de tuples dont le premier élément et un itemset trouvé et le deuxième élément est la valeur de support correspondante.

In [53]:
def apriori(base, minsup):
    unionF=[]
    c = singletons(base)
    B=len(base)
    while c:
        count=[]
        support=[]
        for ci in c:
            count.append(comptage(base, ci))
        f=[]
        for i in range(len(c)):
            if count[i]/B >= minsup:
                f.append(c[i])
                support.append(count[i]/B)
                
        c=apriori_gen(f)
        for i in range(len(f)):
            unionF.append((f[i],support[i]))
            
    return unionF

In [54]:
print("Exemple: apriori(Base1, 0.3]) rend ")
print(apriori(Base1,0.3))

Exemple: apriori(Base1, 0.3]) rend 
[({'b'}, 0.7), ({'c'}, 0.7), ({'a'}, 0.6), ({'e'}, 0.3), ({'d'}, 0.7), ({'b', 'c'}, 0.6), ({'b', 'a'}, 0.5), ({'b', 'd'}, 0.4), ({'a', 'c'}, 0.4), ({'c', 'd'}, 0.4), ({'a', 'd'}, 0.4), ({'e', 'd'}, 0.3), ({'b', 'c', 'a'}, 0.4), ({'b', 'c', 'd'}, 0.3), ({'b', 'a', 'd'}, 0.3)]


In [55]:
print("Exemple: apriori(Base1, 0.3]) rend ")
print(apriori(Base1,0.3))

Exemple: apriori(Base1, 0.3]) rend 
[({'b'}, 0.7), ({'c'}, 0.7), ({'a'}, 0.6), ({'e'}, 0.3), ({'d'}, 0.7), ({'b', 'c'}, 0.6), ({'b', 'a'}, 0.5), ({'b', 'd'}, 0.4), ({'a', 'c'}, 0.4), ({'c', 'd'}, 0.4), ({'a', 'd'}, 0.4), ({'e', 'd'}, 0.3), ({'b', 'c', 'a'}, 0.4), ({'b', 'c', 'd'}, 0.3), ({'b', 'a', 'd'}, 0.3)]


## Expérimentations

### Vérification avec la base du TD

Utiliser la fonction <code>apriori</code> avec la base de transactions du TD afin de vérifier que les itemsets obtenus sont corrects.

In [56]:
BaseTD={1: {'a', 'b', 'd','e'},
 2: {'a','b', 'd', 'e'},
 3: {'b', 'c', 'd'},
 4: {'a', 'e', 'c', 'd'},
 5: {'b', 'c','d','e'},
 6: {'e', 'b', 'd'},
 7: {'d', 'c'},
 8: {'a', 'b', 'c'},
 9: {'a', 'd', 'e'},
 10: { 'b', 'd'}}

In [61]:
print("Exemple: apriori(BaseTD, 0.3) rend ")
print(apriori(BaseTD,0.3))

Exemple: apriori(BaseTD, 0.3) rend 
[({'b'}, 0.7), ({'c'}, 0.5), ({'a'}, 0.5), ({'e'}, 0.6), ({'d'}, 0.9), ({'b', 'c'}, 0.3), ({'b', 'a'}, 0.3), ({'b', 'e'}, 0.4), ({'b', 'd'}, 0.6), ({'c', 'd'}, 0.4), ({'a', 'e'}, 0.4), ({'a', 'd'}, 0.4), ({'e', 'd'}, 0.6), ({'b', 'e', 'd'}, 0.4), ({'a', 'e', 'd'}, 0.4)]


### Comparaisons avec les implémentations apriori et fpgrowth

<font color="RED" size="+1">**[Q]**</font>  Tester votre fonction <code>apriori</code> sur les 3 bases données dans datasets et confronter les résultats obtenus avec ceux fournis par l'exécutable <code>apriori</code>.

In [67]:
print("Résultat du chargement de 'mushrooms.txt', on obtient : ")
Base_mushrooms = chargeBase('datasets/mushrooms.txt',delimiteur=',')
#Base_mushrooms
print(len(Base_mushrooms),' transactions.')

Résultat du chargement de 'mushrooms.txt', on obtient : 
8416  transactions.


In [69]:
print("Pour la BASE précédente :")
noms_items(Base_mushrooms)
print("Exemple: singletons(Base_mushrooms) rend :")
singletons(Base_mushrooms)

Pour la BASE précédente :
Exemple: singletons(Base_mushrooms) rend :


[{'TAPERING'},
 {'CONVEX'},
 {'FIBROUS'},
 {'ORANGE'},
 {'PATHS'},
 {'PINK'},
 {'WHITE'},
 {'ALMOND'},
 {'SCALY'},
 {'YELLOW'},
 {'SCATTERED'},
 {'BROAD'},
 {'ONE'},
 {'CLUSTERED'},
 {'GRASSES'},
 {'NONE'},
 {'PUNGENT'},
 {'BROWN'},
 {'GRAY'},
 {'FOUL'},
 {'GROOVES'},
 {'WOODS'},
 {'PURPLE'},
 {'SUNKEN'},
 {'FREE'},
 {'ATTACHED'},
 {'NUMEROUS'},
 {'BRUISES'},
 {'CROWDED'},
 {'TWO'},
 {'PENDANT'},
 {'CONICAL'},
 {'SILKY'},
 {'FISHY'},
 {'NARROW'},
 {'CHOCOLATE'},
 {'BUFF'},
 {'PARTIAL'},
 {'WASTE'},
 {'ROOTED'},
 {'GREEN'},
 {'SMOOTH'},
 {'KNOBBED'},
 {'EDIBLE'},
 {'POISONOUS'},
 {'LARGE'},
 {'SEVERAL'},
 {'ENLARGING'},
 {'SPICY'},
 {'MUSTY'},
 {'EQUAL'},
 {'SOLITARY'},
 {'NO'},
 {'EVANESCENT'},
 {'ABUNDANT'},
 {'ANISE'},
 {'CREOSOTE'},
 {'BLACK'},
 {'MEADOWS'},
 {'BULBOUS'},
 {'URBAN'},
 {'CLUB'},
 {'CLOSE'},
 {'RED'},
 {'?'},
 {'FLAT'},
 {'CINNAMON'},
 {'BELL'},
 {'FLARING'},
 {'LEAVES'}]

In [21]:
print(apriori(Base_mushrooms,0.9))

[({'WHITE'}, 0.9771863117870723), ({'PARTIAL'}, 1.0), ({'FREE'}, 0.9743346007604563), ({'ONE'}, 0.9230038022813688), ({'WHITE', 'PARTIAL'}, 0.9771863117870723), ({'WHITE', 'FREE'}, 0.9743346007604563), ({'WHITE', 'ONE'}, 0.9001901140684411), ({'PARTIAL', 'FREE'}, 0.9743346007604563), ({'PARTIAL', 'ONE'}, 0.9230038022813688), ({'FREE', 'ONE'}, 0.9001901140684411), ({'WHITE', 'PARTIAL', 'FREE'}, 0.9743346007604563), ({'WHITE', 'PARTIAL', 'ONE'}, 0.9001901140684411), ({'WHITE', 'FREE', 'ONE'}, 0.9001901140684411), ({'PARTIAL', 'FREE', 'ONE'}, 0.9001901140684411), ({'FREE', 'WHITE', 'PARTIAL', 'ONE'}, 0.9001901140684411)]


In [79]:
[({'WHITE'}, 0.9771863117870723), ({'PARTIAL'}, 1.0), ({'FREE'}, 0.9743346007604563), ({'ONE'}, 0.9230038022813688), ({'WHITE', 'PARTIAL'}, 0.9771863117870723), ({'WHITE', 'FREE'}, 0.9743346007604563), ({'WHITE', 'ONE'}, 0.9001901140684411), ({'PARTIAL', 'FREE'}, 0.9743346007604563), ({'PARTIAL', 'ONE'}, 0.9230038022813688), ({'FREE', 'ONE'}, 0.9001901140684411), ({'WHITE', 'PARTIAL', 'FREE'}, 0.9743346007604563), ({'WHITE', 'PARTIAL', 'ONE'}, 0.9001901140684411), ({'WHITE', 'FREE', 'ONE'}, 0.9001901140684411), ({'PARTIAL', 'FREE', 'ONE'}, 0.9001901140684411), ({'FREE', 'WHITE', 'PARTIAL', 'ONE'}, 0.9001901140684411)]

[({'WHITE'}, 0.9771863117870723),
 ({'PARTIAL'}, 1.0),
 ({'FREE'}, 0.9743346007604563),
 ({'ONE'}, 0.9230038022813688),
 ({'PARTIAL', 'WHITE'}, 0.9771863117870723),
 ({'FREE', 'WHITE'}, 0.9743346007604563),
 ({'ONE', 'WHITE'}, 0.9001901140684411),
 ({'FREE', 'PARTIAL'}, 0.9743346007604563),
 ({'ONE', 'PARTIAL'}, 0.9230038022813688),
 ({'FREE', 'ONE'}, 0.9001901140684411),
 ({'FREE', 'PARTIAL', 'WHITE'}, 0.9743346007604563),
 ({'ONE', 'PARTIAL', 'WHITE'}, 0.9001901140684411),
 ({'FREE', 'ONE', 'WHITE'}, 0.9001901140684411),
 ({'FREE', 'ONE', 'PARTIAL'}, 0.9001901140684411),
 ({'FREE', 'ONE', 'PARTIAL', 'WHITE'}, 0.9001901140684411)]

In [80]:
print("Résultat du chargement de 'titanic-red.csv', on obtient : ")
Base_titanic = chargeBase('datasets/titanic-red.csv',delimiteur=',')
print(len(Base_titanic),' transactions.')

Résultat du chargement de 'titanic-red.csv', on obtient : 
741  transactions.


In [82]:
# décommenter les lignes suivantes:

print("Pour la BASE Titanic :")
print(noms_items(Base_titanic))

print("Exemple: singletons(Base_titanic) rend :")
singletons(Base_titanic)

Pour la BASE Titanic :
{'BroomfieldChelmsfordEngland', 'EnglandSaltLakeCityUtah', 'AustriaHungaryGermantownPhiladelphiaPA', 'BrightonSussex', 'Belfast', 'BrooklynNY', 'ZurichSwitzerland', 'NewYorkNYStamfordCT', 'Paris', 'BrightonMA', 'PhiladelphiaPA', 'SkaraSwedenRockfordIL', 'ParisFranceNewYorkNY', 'EnglandOglesbyIL', 'NewYorkNYGreenwichCT', 'LondonEastOrangeNJ', 'SarniaON', 'FondduLacWI', 'ElmiraNYOrangeNJ', 'MerrillWI', 'embarked', 'BristolEnglandClevelandOH', 'BelgiumDetroitMI', 'ToftaSwedenJolietIL', 'DenmarkHillSurreyChicago', 'KingwilliamstownCoCorkIrelandNewYorkNY', 'FrankfortKY', 'TrentonNJ', 'HelsinkiFinlandAshtabulaOhio', 'EnglandNewYorkNY', 'RotterdamNetherlands', 'WaukeganChicagoIL', 'HarrowEngland', 'MamaroneckNY', 'GreenBayWI', 'PlymouthEngland', 'DevonEnglandWichitaKS', 'LimaPeru', 'BathEnglandMassachusetts', 'DowagiacMI', 'RochesterNY', 'MexicoCityMexico', '1st', 'TreherbertCardiffWales', 'FinlandMinneapolisMN', 'Liverpool', 'CornwallAkronOH', 'BerneSwitzerlandCentralC

[{'BroomfieldChelmsfordEngland'},
 {'EnglandSaltLakeCityUtah'},
 {'AustriaHungaryGermantownPhiladelphiaPA'},
 {'BrightonSussex'},
 {'Belfast'},
 {'BrooklynNY'},
 {'ZurichSwitzerland'},
 {'NewYorkNYStamfordCT'},
 {'Paris'},
 {'BrightonMA'},
 {'PhiladelphiaPA'},
 {'SkaraSwedenRockfordIL'},
 {'ParisFranceNewYorkNY'},
 {'EnglandOglesbyIL'},
 {'NewYorkNYGreenwichCT'},
 {'LondonEastOrangeNJ'},
 {'SarniaON'},
 {'FondduLacWI'},
 {'ElmiraNYOrangeNJ'},
 {'MerrillWI'},
 {'embarked'},
 {'BristolEnglandClevelandOH'},
 {'BelgiumDetroitMI'},
 {'ToftaSwedenJolietIL'},
 {'DenmarkHillSurreyChicago'},
 {'KingwilliamstownCoCorkIrelandNewYorkNY'},
 {'FrankfortKY'},
 {'TrentonNJ'},
 {'HelsinkiFinlandAshtabulaOhio'},
 {'EnglandNewYorkNY'},
 {'RotterdamNetherlands'},
 {'WaukeganChicagoIL'},
 {'HarrowEngland'},
 {'MamaroneckNY'},
 {'GreenBayWI'},
 {'PlymouthEngland'},
 {'DevonEnglandWichitaKS'},
 {'LimaPeru'},
 {'BathEnglandMassachusetts'},
 {'DowagiacMI'},
 {'RochesterNY'},
 {'MexicoCityMexico'},
 {'1st'},
 {

In [85]:
Lres = apriori(Base_titanic,0.2)

# tri sur les valeurs de supports:
#print(sorted(Lres, key=lambda data: data[1]))

# tri sur la taille des itemsets
print(sorted(Lres, key=lambda data: len(data[0])))

for t in sorted(Lres, key=lambda data: data[1],reverse = True):
    if len(t[0])>1:
        print(t)


[({'1st'}, 0.3792172739541161), ({'dead'}, 0.5344129554655871), ({'female'}, 0.38866396761133604), ({'3rd'}, 0.27125506072874495), ({'Southampton'}, 0.7112010796221323), ({'2nd'}, 0.3481781376518219), ({'Cherbourg'}, 0.23751686909581646), ({'alive'}, 0.4642375168690958), ({'male'}, 0.6099865047233468), ({'Southampton', '1st'}, 0.20917678812415655), ({'1st', 'alive'}, 0.23076923076923078), ({'1st', 'male'}, 0.21997300944669365), ({'Southampton', 'dead'}, 0.41295546558704455), ({'male', 'dead'}, 0.46963562753036436), ({'female', 'Southampton'}, 0.2483130904183536), ({'female', 'alive'}, 0.32388663967611336), ({'Southampton', '2nd'}, 0.30634278002699056), ({'Southampton', 'alive'}, 0.2982456140350877), ({'Southampton', 'male'}, 0.46288798920377866), ({'2nd', 'male'}, 0.20647773279352227), ({'Southampton', 'male', 'dead'}, 0.3684210526315789), ({'female', 'alive', 'Southampton'}, 0.203778677462888)]
({'male', 'dead'}, 0.46963562753036436)
({'Southampton', 'male'}, 0.46288798920377866)
({'S

<font color="RED" size="+1">**[Q]**</font>  Utiliser l'éxécutable <code>apriori</code> pour générer des règles d'association (voir les différentes options en annexe) sur les différentes bases de données fournies. Tester différents seuils (support et confiance). 

In [89]:
#./apriori32 -trs50m2n5 datasets/exemple-1.txt fichier-resultat.out

c <- a b (50, 80)
b <- a (60, 83.3333)
c <- b (70, 85.7143)
b <- c (70, 85.7143)

SyntaxError: invalid syntax (<ipython-input-89-c0e4e662a98e>, line 3)

<font color="RED" size="+1">**[Q]**</font>   Ajouter les deux mesures d'intérêt vues en TD (lift et RR) et afficher leurs valeurs pour chaque règle trouvée. Ajouter un argument au programme afin de pouvoir sélectionner une de ces mesures pour éliminer les règles inintéressantes.


<font color="RED" size="+1">**[Q]**</font>  Ajouter les deux mesures d'intérêt suivantes et afficher leurs valeurs
  pour chaque règle trouvée. Ajouter un argument au programme afin de pouvoir sélectionner une de ces mesures pour éliminer les règles inintéressantes.
  
Par exemple:

\begin{align}
  \mbox{Interest}(X \longrightarrow Y) & =  \frac{P(X,Y)}{P(X)}P(Y) \nonumber\\
 \mbox{IS}(X \longrightarrow Y) & =  \frac{P(X,Y)}{\sqrt{P(X)P(Y)}} \nonumber
\end{align}

### Utilisation de fp-growth

<font color="RED" size="+1">**[Q]**</font>   Utiliser <code>fpgrowth</code> pour générer des règles d'association sur les différentes
  bases de données. Comparer avec les résultats obtenus dans la section précédente. En  particulier, comparer les temps d'éxecution.

In [90]:
#./fpgrowth32 -trs50m2n5 datasets/exemple-1.txt fichier-resultat.out

b <- c (70, 85.7143)
c <- b (70, 85.7143)
b <- a (60, 83.3333)
c <- a b (50, 80)


SyntaxError: invalid syntax (<ipython-input-90-c04d5c8f5d82>, line 6)

<font color="RED" size="+1">**[Q]**</font>  En essayant différentes valeurs de seuil, tester la génération de règles d'association intéressantes sur les bases de données fournies.

In [ ]:
#seuil a 20
b <- c (70, 85.7143)
c <- b (70, 85.7143)
b <- a (60, 83.3333)
b <- a c (40, 100)
c <- a b (50, 80)
b <- a d c (20, 100)
d <- e (30, 100)


## Annexes: utilisation des exécutables *apriori* et *fpgrowth*

La documentation complète de ces programmes est disponible aux URL suivantes:
- pour *apriori*: http://www.borgelt.net/doc/apriori/apriori.html
- pour *fp-growth*: http://www.borgelt.net/doc/fpgrowth/fpgrowth.html


Leur format d'utilisation général est:

        ./<programme> [options] infile [outfile]

(où <<code>programme</code>> est soit *apriori*, soit *fpgrowth*).

Les options de base de ces 2 programmes sont:
- sans argument: génération des itemsets fréquents (argument {<code>-ts</code> activé par défaut)
- *tr*: pour obtenir les règles d'associations
- *s*: pour fournir une valeur minimale de support. Le support est ici donné en valeur absolue ($n_{AB}$) et non pas en valeur relative ($\frac{n_{AB}}{n}$).
- *m*: pour fournir un nombre minimum d'items dans un itemset
- *n*: pour fournir un nombre maximum d'items dans un itemset


Par exemple (commandes lancées dans le répertoire père du répertoire *datasets/*):

        ./apriori -trs50m2n5 datasets/exemple-1.txt fichier-resultat.out  
        ./fpgrowth -trs50m2n5 datasets/mushrooms.txt fichier-resultat.out  
